In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

Mounted at /content/drive


##환경설정

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.0 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661677 sha256=77782a24081a3c02065a924caccbaa1118ff129812ef8aa1b4c0bbefe8fcd8b8
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.8 MB/s eta 0:00:00


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-tjbr68fw/kobert-tokenizer_d71fa65b1a2244c4a9f6519d7cc7173e
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-tjbr68fw/kobert-tokenizer_d71fa65b1a2244c4a9f6519d7cc7173e
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=d709b4cf2d5fdc06c432d854fb83e09bdbbb7b07f6d8d02dad9b759ee2a0536b
  Stored in directory: /tmp/pip-ephem-wheel-cache-1x2ldvqm/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

##데이터셋 전처리

In [ ]:
import pandas as pd

# CSV 파일 경로 설정
csv_file_path = '/content/drive/MyDrive/ML/Project/emotion_data_final.csv'

# CSV 파일을 DataFrame으로 로드
df = pd.read_csv(csv_file_path)

# DataFrame 내용 확인
data = df

data

,emotion,sentence
0,화,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...
1,화,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,화,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,화,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,화,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...
...,...,...
46416,슬픔,나이가 다 돼서 퇴직을 하게 되었어. 하지만 노후 준비를 제대로 안 해서 돈 걱정이...
46417,스트레스,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....
46418,걱정,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나. 마음...
46419,걱정,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가...


In [ ]:
data.loc[(data['emotion'] == "화"), 'emotion'] = 0
data.loc[(data['emotion'] == "편안"), 'emotion'] = 1
data.loc[(data['emotion'] == "걱정"), 'emotion'] = 2
data.loc[(data['emotion'] == "당황"), 'emotion'] = 3
data.loc[(data['emotion'] == "만족"), 'emotion'] = 4
data.loc[(data['emotion'] == "괴로움"), 'emotion'] = 5
data.loc[(data['emotion'] == "후회"), 'emotion'] = 6
data.loc[(data['emotion'] == "슬픔"), 'emotion'] = 7
data.loc[(data['emotion'] == "스트레스"), 'emotion'] = 8
data.loc[(data['emotion'] == "신뢰"), 'emotion'] = 9
data.loc[(data['emotion'] == "신남"), 'emotion'] = 10
data.loc[(data['emotion'] == "실망"), 'emotion'] = 11
data.loc[(data['emotion'] == "안도"), 'emotion'] = 12
data.loc[(data['emotion'] == "억울"), 'emotion'] = 13
data.loc[(data['emotion'] == "열등감"), 'emotion'] = 14
data.loc[(data['emotion'] == "회의적"), 'emotion'] = 15
data.loc[(data['emotion'] == "외로운"), 'emotion'] = 16
data.loc[(data['emotion'] == "혐오"), 'emotion'] = 17
data.loc[(data['emotion'] == "자신감"), 'emotion'] = 18
data.loc[(data['emotion'] == "기쁨"), 'emotion'] = 19
data.loc[(data['emotion'] == "충격"), 'emotion'] = 20
data.loc[(data['emotion'] == "질투"), 'emotion'] = 21
data.loc[(data['emotion'] == "감사"), 'emotion'] = 22

data

,emotion,sentence
0,0,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...
1,0,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,0,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,0,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,0,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...
...,...,...
46416,7,나이가 다 돼서 퇴직을 하게 되었어. 하지만 노후 준비를 제대로 안 해서 돈 걱정이...
46417,8,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....
46418,2,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나. 마음...
46419,2,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가...


In [ ]:
# 필요한 데이터를 data_list 배열에 저장
data_list = []
for sentence, label in zip(data['sentence'], data['emotion']):
    data = []
    data.append(sentence)
    data.append(str(label))
    data_list.append(data)

for data in data_list:
    print(data)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
['지금 남자친구와 결혼을 약속하고 준비 중이야. 근데 같이 살 집이랑 차를 마련해야 하는데 경제적으로 힘들어서 걱정이야.', '2']
['이번에 시험을 망쳐서 짜증 나. 다른 친구들은 잘 본 것 같아서 나한테 더 화가 나. 노래방 가서 노래를 부르고 오면 스트레스가 풀릴 것 같아.', '0']
['반 친구 중 한 명이 나를 자주 괴롭혀서 선생님께 말씀드렸는데 도리어 나에게 짜증을 내셔. 담임 선생님 말고 학생부 선생님께 말씀드려 봐야겠어.', '8']
['엄마가 자꾸 공부에 참견해서 미치겠어. 잔소리 안 하고 그냥 내버려 뒀으면 좋겠어. 그냥 짜증만 냈어. 이야기해 보는 게 좋을 것 같아.', '8']
['학교 가는 게 즐겁지 않아. 너무 가기 싫어. 친구들 때문에 가기 싫어. 이유도 없이 괴롭혀서 미치겠어.', '0']
['오늘 시험을 망쳤어. 그래서 정말 화가 나. 응. 공부를 정말 열심히 했는데 실수를 했어. 다음에는 문제를 더 꼼꼼히 읽고 한순간도 안심해서는 안 될 것 같아. 다음 시험은 꼭 잘 보고 싶어.', '0']
['이제 다른 사람들을 대하는 게 너무 힘들어. 인간관계에 상처를 너무 많이 받은 거 같아. 중학교 때 받았던 학교폭력과 따돌림도 그렇고.', '14']
['부모님이랑 오늘도 다퉜어. 심한 말을 들으니 무기력해져. 저번 학기보다 내 성적이 너무 떨어져서 그런 것 같아. 이번 시험은 더 잘 봐야겠어. 예습과 복습을 철저히 해야지.', '11']
['성적도 낮고 공부도 못하는데 내 진로를 뭐로 결정해야 하지? 진짜 짜증 나. 그것도 잘 모르겠는데. 이번 기회에 적성 검사나 성격 검사를 받아보는 것도 괜찮겠다. 검사 결과 나는 친화력이 좋고 낯을 가리지 않아서 서비스직이 적합하다고 나왔어.', '8']
['오늘 원했던 직장 최종 면접을 봤는데 너무 못 본 것 같아 너무 속상해. 면접관분들께 잘 보이기 위해 수백 번 연습했는데 상처를 받았어.', '7']
['최종 면접에서 면접관

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# train data와 test data로 나누기
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=32)

print(len(dataset_train)) # 34815 출력
print(len(dataset_test)) # 11606 출력

37136
9285


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 128
batch_size = 16
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=23,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-30-e6a38b13095b>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/2321 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.186347007751465 train acc 0.0625
epoch 1 batch id 201 loss 2.8012588024139404 train acc 0.10851990049751244
epoch 1 batch id 401 loss 2.4304025173187256 train acc 0.1667705735660848
epoch 1 batch id 601 loss 2.3866612911224365 train acc 0.2074667221297837
epoch 1 batch id 801 loss 2.1105382442474365 train acc 0.2381398252184769
epoch 1 batch id 1001 loss 1.7821564674377441 train acc 0.2642982017982018
epoch 1 batch id 1201 loss 2.0359091758728027 train acc 0.28700041631973355
epoch 1 batch id 1401 loss 1.748673439025879 train acc 0.3057637401855817
epoch 1 batch id 1601 loss 2.3300302028656006 train acc 0.3228060587133042
epoch 1 batch id 1801 loss 1.8430047035217285 train acc 0.33637562465297055
epoch 1 batch id 2001 loss 2.1046531200408936 train acc 0.3474512743628186
epoch 1 batch id 2201 loss 0.9275615215301514 train acc 0.3604327578373467
epoch 1 train acc 0.3664368806548901


<ipython-input-30-e6a38b13095b>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/581 [00:00<?, ?it/s]

...

epoch 9 test acc 0.5364457831325301


  0%|          | 0/2321 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.05027065426111221 train acc 1.0
epoch 10 batch id 201 loss 0.5142224431037903 train acc 0.912002487562189
epoch 10 batch id 401 loss 0.3280397355556488 train acc 0.9192643391521197
epoch 10 batch id 601 loss 0.0925501361489296 train acc 0.9169093178036606
epoch 10 batch id 801 loss 0.4099193811416626 train acc 0.915496254681648
epoch 10 batch id 1001 loss 0.4680405557155609 train acc 0.9150849150849151
epoch 10 batch id 1201 loss 0.0451955609023571 train acc 0.9149666944213156
epoch 10 batch id 1401 loss 0.6765365600585938 train acc 0.9139007851534618
epoch 10 batch id 1601 loss 0.45575442910194397 train acc 0.9138038725796377
epoch 10 batch id 1801 loss 0.02946075238287449 train acc 0.9129650194336479
epoch 10 batch id 2001 loss 0.09149385243654251 train acc 0.9136994002998501
epoch 10 batch id 2201 loss 0.026578543707728386 train acc 0.915010222626079
epoch 10 train acc 0.9156074967686342


  0%|          | 0/581 [00:00<?, ?it/s]

epoch 10 test acc 0.538919965576592


In [ ]:
import os
import torch

# Define the directory to save the model files
output_dir = "/content/drive/MyDrive/ML/Project/result"

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save the final trained model with a specific file path
final_model_file_path = os.path.join(output_dir, 'final_model_epoch10.pt')
torch.save(model.state_dict(), final_model_file_path)

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Hugging Face의 BERT 모델과 토크나이저 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
todel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)

# 저장된 모델 가중치 불러오기
saved_model_path = "/content/drive/MyDrive/ML/Project/result/final_model_epoch10.pt"
model.load_state_dict(torch.load(saved_model_path))

# 모델을 evaluation mode로 설정 (필요에 따라)
model.eval()



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another,  0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            if np.argmax(logits) == 0:
              test_eval.append("화")
            elif np.argmax(logits) == 1:
              test_eval.append("편안")
            elif np.argmax(logits) == 2:
              test_eval.append("걱정")
            elif np.argmax(logits) == 3:
              test_eval.append("당황")
            elif np.argmax(logits) == 4:
              test_eval.append("만족")
            elif np.argmax(logits) == 5:
              test_eval.append("괴로움")
            elif np.argmax(logits) == 6:
              test_eval.append("후회")
            elif np.argmax(logits) == 7:
              test_eval.append("슬픔")
            elif np.argmax(logits) == 8:
              test_eval.append("스트레스")
            elif np.argmax(logits) == 9:
              test_eval.append("신뢰")
            elif np.argmax(logits) == 10:
              test_eval.append("신남")
            elif np.argmax(logits) == 11:
              test_eval.append("실망")
            elif np.argmax(logits) == 12:
              test_eval.append("안도")
            elif np.argmax(logits) == 13:
              test_eval.append("억울")
            elif np.argmax(logits) == 14:
              test_eval.append("열등감")
            elif np.argmax(logits) == 15:
              test_eval.append("회의적")
            elif np.argmax(logits) == 16:
              test_eval.append("외로운")
            elif np.argmax(logits) == 17:
              test_eval.append("혐오")
            elif np.argmax(logits) == 18:
              test_eval.append("자신감")
            elif np.argmax(logits) == 19:
              test_eval.append("기쁨")
            elif np.argmax(logits) == 20:
              test_eval.append("충격")
            elif np.argmax(logits) == 21:
              test_eval.append("질투")
            elif np.argmax(logits) == 22:
              test_eval.append("감사")

        print(test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")